# Delhi Colonies Public Services Index

* Load in colonies dataset from Pickle file `colonies_10Aug2020.data` **[done]**
    * Create two versions of colonies, with one based on touching vs. bbox neighbors
    * Give each version a distinct name
* Import services shapefiles **[done]**
    * Make sure correct file paths exist
    * Ensure that all shapefiles are valid using `check_shapefile` function
    * Reproject shapefiles to EPSG 7760 (if needed)
* Compute all Services indices (turn into a function)
    * Compute Point Services Indices
    * Compute Roads Index (make sure that units are in kilometers)
    * Compute average of all service indices
    * Run services indices twice (on touching vs. bbox neighbors) using function
* Compute simple statistics (mean, min, max) based on colony type
    * Turn into function
    * Run on both datasets

## Import modules and set constants

In [1]:
import os
import pickle
from importlib import reload
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, box
import spatial_index_utils

In [2]:
reload(spatial_index_utils)

<module 'spatial_index_utils' from 'C:\\Users\\bwbel\\Google Drive\\slum_project\\spatial_index_python\\spatial_index_utils.py'>

In [3]:
# WGS 84 / Delhi
epsg_code = 7760

## Load colonies dataset and create two versions

In [4]:
with open('colonies_10Aug2020.data', 'rb') as f:
    colonies = pickle.load(f)

In [5]:
colonies_touch = colonies.drop(columns=['nbrs_bbox', 'nbrs_dist_bbox'])

In [6]:
colonies_touch.head()

,AREA,USO_AREA_U,HOUSETAX_C,USO_FINAL,geometry,canal,railway,drain,barrier,nbrs_touch,centroid,nbrs_dist_touch,ndmc_dist_km,area_km2,index,population
0,Singhola,3058,H,RV,"POLYGON Z ((1013763.588 1023721.838 0.000, 101...",False,False,True,True,[],POINT (1012643.931 1023967.383),[],24.721016,2.733999,0,4415.586042
1,Indra Colony (Narela),1760,G,RUAC,"POLYGON Z ((1007997.730 1025421.961 0.000, 100...",False,True,False,True,"[2869, 5445]",POINT (1008331.501 1025234.188),"[(2869, 0.5868185653539563), (5445, 0.31827133...",27.676855,0.182393,1,3990.707763
2,Bhor Garh,1276,H,Industrial,"POLYGON Z ((1008543.236 1022671.585 0.000, 100...",False,True,False,True,[5478],POINT (1009672.909 1022802.723),"[(5478, 0.661418841164336)]",24.900046,1.470808,2,6849.523714
3,Gautam Colony,1528,G,RUAC,"POLYGON Z ((1008080.674 1025132.190 0.000, 100...",False,True,False,True,"[2869, 5095, 5105, 5436, 5457]",POINT (1008902.120 1025098.259),"[(2869, 0.5676799818012491), (5095, 0.32048263...",27.289733,1.150775,3,26712.165262
4,Kureni,2082,H,RV,"POLYGON Z ((1009508.695 1025281.671 0.000, 100...",False,True,False,True,"[3963, 5092, 5098, 5104, 5106, 5451]",POINT (1009491.379 1023952.320),"[(3963, 0.6363487153623809), (5092, 1.29943169...",26.001177,2.415990,4,28819.392920


In [7]:
colonies_bbox = colonies.drop(columns=['nbrs_touch', 'nbrs_dist_touch'])

In [8]:
colonies_bbox.head()

,AREA,USO_AREA_U,HOUSETAX_C,USO_FINAL,geometry,canal,railway,drain,barrier,nbrs_bbox,centroid,nbrs_dist_bbox,ndmc_dist_km,area_km2,index,population
0,Singhola,3058,H,RV,"POLYGON Z ((1013763.588 1023721.838 0.000, 101...",False,False,True,True,"[5108, 5487, 5488]",POINT (1012643.931 1023967.383),"[(5108, 1.4513506146004875), (5487, 1.17056023...",24.721016,2.733999,0,4415.586042
1,Indra Colony (Narela),1760,G,RUAC,"POLYGON Z ((1007997.730 1025421.961 0.000, 100...",False,True,False,True,"[2869, 5445]",POINT (1008331.501 1025234.188),"[(2869, 0.5868185653539563), (5445, 0.31827133...",27.676855,0.182393,1,3990.707763
2,Bhor Garh,1276,H,Industrial,"POLYGON Z ((1008543.236 1022671.585 0.000, 100...",False,True,False,True,"[4270, 4273, 5478]",POINT (1009672.909 1022802.723),"[(4270, 0.8033104667179346), (4273, 0.58109669...",24.900046,1.470808,2,6849.523714
3,Gautam Colony,1528,G,RUAC,"POLYGON Z ((1008080.674 1025132.190 0.000, 100...",False,True,False,True,"[2869, 3963, 4243, 5092, 5095, 5105, 5436, 544...",POINT (1008902.120 1025098.259),"[(2869, 0.5676799818012491), (3963, 0.70232814...",27.289733,1.150775,3,26712.165262
4,Kureni,2082,H,RV,"POLYGON Z ((1009508.695 1025281.671 0.000, 100...",False,True,False,True,"[3963, 5092, 5095, 5098, 5104, 5106, 5451, 5478]",POINT (1009491.379 1023952.320),"[(3963, 0.6363487153623809), (5092, 1.29943169...",26.001177,2.415990,4,28819.392920


## Import services shapefiles

In [9]:
# Define filepaths

services_dir = os.path.join('shapefiles', 'Spatial_Index_GIS', 'Public Services')

bank_fp = os.path.join(services_dir, 'Banking', 'Banking.shp')
health_fp = os.path.join(services_dir, 'Health', 'Health.shp')
road_fp = os.path.join(services_dir, 'Major Road', 'Road.shp')
police_fp = os.path.join(services_dir, 'Police', 'Police Station.shp')
ration_fp = os.path.join(services_dir, 'Ration', 'Ration.shp')
school_fp = os.path.join(services_dir, 'School', 'schools7760.shp')
transport_fp = os.path.join(services_dir, 'Transport', 'Transport.shp')

# boundary of Delhi
delhi_bounds_filepath = os.path.join('shapefiles', 'delhi_bounds_buffer.shp')

# Check that all filepaths exist
filepath_list = [bank_fp, health_fp, road_fp, police_fp, ration_fp, school_fp, transport_fp, delhi_bounds_filepath]

for filepath in filepath_list:
    if not os.path.exists(filepath):
        print('{} does not exist'.format(filepath))

In [10]:
# Import services
bank = gpd.read_file(bank_fp)
health = gpd.read_file(health_fp)
road = gpd.read_file(road_fp)
police = gpd.read_file(police_fp)
ration = gpd.read_file(ration_fp)
school = gpd.read_file(school_fp)
transport = gpd.read_file(transport_fp)

## Check validity of services shapefiles
* Duplicate rows are okay for ATMs (I assume that ATM locations for the same bank in a similar location will seem to be counted twice)
* Look specifically for invalid geometries and whether shapefile is fully contained within Delhi

In [11]:
spatial_index_utils.check_shapefile(gdf=bank, gdf_name='bank', 
                                    geom_type='Point', 
                                    delhi_bounds_filepath=delhi_bounds_filepath)

bank has duplicate rows: True
----------------------------------------------------
rows with invalid geometries 

----------------------------------------------------
all geometries in bank are of type Point: True
----------------------------------------------------
Rows with None value in geometry column are below
Empty GeoDataFrame
Columns: [bank_name, Latitude, Longitude, Type, geometry, geom_type]
Index: []
----------------------------------------------------
bank shapefile is contained within Delhi: True
----------------------------------------------------
Done with shapefile evaluation


In [12]:
spatial_index_utils.check_shapefile(gdf=health, gdf_name='health', 
                                    geom_type='Point', 
                                    delhi_bounds_filepath=delhi_bounds_filepath)

health has duplicate rows: False
----------------------------------------------------
rows with invalid geometries 

----------------------------------------------------
all geometries in health are of type Point: True
----------------------------------------------------
Rows with None value in geometry column are below
Empty GeoDataFrame
Columns: [Hospital_C, ADDRESS, X, Y, geometry, geom_type]
Index: []
----------------------------------------------------
health shapefile is contained within Delhi: True
----------------------------------------------------
Done with shapefile evaluation


In [13]:
spatial_index_utils.check_shapefile(gdf=road, gdf_name='road', 
                                    geom_type='Line', 
                                    delhi_bounds_filepath=delhi_bounds_filepath)

road has duplicate rows: False
----------------------------------------------------
rows with invalid geometries 

----------------------------------------------------
all geometries in road are of type Line: True
----------------------------------------------------
Rows with None value in geometry column are below
Empty GeoDataFrame
Columns: [FID, RD_NM, RD_CLS, RD_LANES, RD_TP_SRF, RD_MB, RD_ONEWAY, EL_GND, DIST_NM, ONEWAY, Speed_kmph, geometry, geom_type]
Index: []
----------------------------------------------------
road shapefile is contained within Delhi: True
----------------------------------------------------
Done with shapefile evaluation


In [14]:
spatial_index_utils.check_shapefile(gdf=police, gdf_name='police', 
                                    geom_type='Point', 
                                    delhi_bounds_filepath=delhi_bounds_filepath)

police has duplicate rows: False
----------------------------------------------------
rows with invalid geometries 

----------------------------------------------------
all geometries in police are of type Point: True
----------------------------------------------------
Rows with None value in geometry column are below
Empty GeoDataFrame
Columns: [NAME, POLICE_STA, DISTRICT, x, y, geometry, geom_type]
Index: []
----------------------------------------------------
police shapefile is contained within Delhi: True
----------------------------------------------------
Done with shapefile evaluation


In [15]:
spatial_index_utils.check_shapefile(gdf=ration, gdf_name='ration', 
                                    geom_type='Point', 
                                    delhi_bounds_filepath=delhi_bounds_filepath)

ration has duplicate rows: False
----------------------------------------------------
rows with invalid geometries 

----------------------------------------------------
all geometries in ration are of type Point: True
----------------------------------------------------
Rows with None value in geometry column are below
Empty GeoDataFrame
Columns: [S No., License No, FPS ID, Circle, FPS Shop N, Address Of, Latitude, Longitude, Source, geometry, geom_type]
Index: []
----------------------------------------------------
ration shapefile is contained within Delhi: True
----------------------------------------------------
Done with shapefile evaluation


In [16]:
spatial_index_utils.check_shapefile(gdf=school, gdf_name='school', 
                                    geom_type='Point', 
                                    delhi_bounds_filepath=delhi_bounds_filepath)

school has duplicate rows: False
----------------------------------------------------
rows with invalid geometries 

----------------------------------------------------
all geometries in school are of type Point: True
----------------------------------------------------
Rows with None value in geometry column are below
Empty GeoDataFrame
Columns: [objectid_1, objectid, vilname, schname, schcd, schcat, school_cat, pincode, rururb, location, schtype, school_typ, schmgt, management, dtname, stname, stcode11, dtcode11, sdtcode11, sdtname, geometry, geom_type]
Index: []

[0 rows x 22 columns]
----------------------------------------------------
school shapefile is contained within Delhi: True
----------------------------------------------------
Done with shapefile evaluation


In [17]:
spatial_index_utils.check_shapefile(gdf=transport, gdf_name='transport', 
                                    geom_type='Point', 
                                    delhi_bounds_filepath=delhi_bounds_filepath)

transport has duplicate rows: False
----------------------------------------------------
rows with invalid geometries 

----------------------------------------------------
all geometries in transport are of type Point: True
----------------------------------------------------
Rows with None value in geometry column are below
Empty GeoDataFrame
Columns: [stop_id, stop_name, stop_lat, stop_lon, Type, geometry, geom_type]
Index: []
----------------------------------------------------
transport shapefile is contained within Delhi: True
----------------------------------------------------
Done with shapefile evaluation


## Check CRS (all shapefiles should be in EPSG: 7760)

In [18]:
bank.crs == health.crs == road.crs == police.crs == ration.crs == school.crs == transport.crs

True

In [19]:
bank.crs

<Projected CRS: EPSG:7760>
Name: WGS 84 / Delhi
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: India - Delhi
- bounds: (76.83, 28.4, 77.34, 28.89)
Coordinate Operation:
- name: Delhi NSF LCC
- method: Lambert Conic Conformal (2SP)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [20]:
colonies_bbox.crs == colonies_touch.crs == bank.crs

True

## Define Point and Line Services

In [21]:
# Define all point services as dictionary
# makes it easier to calculate all point
# services with one function
point_services = {'bank': bank,
                  'health': health,
                  'police': police,
                  'ration': ration,
                  'school': school,
                  'transport': transport}

line_services = {'road': road}

## Calculate all service indices in one function

In [23]:
from spatial_index_utils import calc_point_services, create_service_length_index

In [24]:
def calc_all_services(polygon_gdf, point_services, line_services, epsg_code, nbr_dist_colname):
    """Calculate all public services indices (point and line)"""
    
    # Get all point services
    polygon_gdf = calc_point_services(polygon_gdf, point_services, epsg_code, nbr_dist_colname)
    
    
    for line_service in line_services:
        polygon_gdf = create_service_length_index(polygon_gdf, 
                                                  line_services[line_service], 
                                                  line_service, 
                                                  epsg_code, 
                                                  nbr_dist_colname)
        
        print('{} service is completed'.format(line_service))
        
    return polygon_gdf

In [25]:
colonies_touch_psi = calc_all_services(colonies_touch, point_services, line_services, epsg_code, 'nbrs_dist_touch')

GeoDataFrame now has the following CRS:

epsg:7760
bank service index is completed
--------------------------------------------------------
GeoDataFrame now has the following CRS:

epsg:7760
health service index is completed
--------------------------------------------------------
GeoDataFrame now has the following CRS:

epsg:7760
police service index is completed
--------------------------------------------------------
GeoDataFrame now has the following CRS:

epsg:7760
ration service index is completed
--------------------------------------------------------
GeoDataFrame now has the following CRS:

epsg:7760
school service index is completed
--------------------------------------------------------
GeoDataFrame now has the following CRS:

epsg:7760
transport service index is completed
--------------------------------------------------------
all point services completed
GeoDataFrame now has the following CRS:

epsg:7760
road service is completed


In [30]:
colonies_touch_psi = colonies_touch_psi.rename(columns={'road_count':'road_length'})
colonies_touch_psi

,AREA,USO_AREA_U,HOUSETAX_C,USO_FINAL,geometry,canal,railway,drain,barrier,nbrs_touch,...,ration_idx,school_count,school_pcen,school_idx,transport_count,transport_pcen,transport_idx,road_length,road_pcen,road_idx
0,Singhola,3058,H,RV,"POLYGON Z ((1013763.588 1023721.838 0.000, 101...",False,False,True,True,[],...,0.001933,1,0.000226,0.001083,8,0.001812,0.003697,5.957213,0.001349,0.006265
1,Indra Colony (Narela),1760,G,RUAC,"POLYGON Z ((1007997.730 1025421.961 0.000, 100...",False,True,False,True,"[2869, 5445]",...,0.000000,0,0.000380,0.001818,2,0.000849,0.001733,0.245861,0.000102,0.000472
2,Bhor Garh,1276,H,Industrial,"POLYGON Z ((1008543.236 1022671.585 0.000, 100...",False,True,False,True,[5478],...,0.000000,3,0.000438,0.002094,4,0.000584,0.001192,1.329173,0.000194,0.000901
3,Gautam Colony,1528,G,RUAC,"POLYGON Z ((1008080.674 1025132.190 0.000, 100...",False,True,False,True,"[2869, 5095, 5105, 5436, 5457]",...,0.001598,7,0.000301,0.001438,7,0.000286,0.000583,0.827590,0.000037,0.000172
4,Kureni,2082,H,RV,"POLYGON Z ((1009508.695 1025281.671 0.000, 100...",False,True,False,True,"[3963, 5092, 5098, 5104, 5106, 5451]",...,0.000889,6,0.000208,0.000996,9,0.000341,0.000697,5.957236,0.000207,0.000960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4285,"HARIJAN BASTI, SADAT PUR, DELHI-94.",5579,None,UAC,"POLYGON Z ((1028449.945 1004222.511 0.000, 102...",False,False,False,False,[],...,0.004376,0,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000
4286,"CHRISTIAN COLONY, PATELCHEST, DELHI-7.",5580,None,UAC,"POLYGON Z ((1004320.138 1007798.133 0.000, 100...",False,False,False,False,[2382],...,0.003527,0,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000
4287,"SULTANPUR MAZRA EXTN,. (EAST), NANGLOI, DELHI-41.",5581,None,UAC,"POLYGON Z ((1004443.956 1007676.189 0.000, 100...",False,False,False,False,[2382],...,0.008620,0,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000
4288,"NAI BASTI BAKNER PASCHIM,DELHI.",5582,None,UAC,"POLYGON Z ((1003152.614 1011361.143 0.000, 100...",False,False,False,False,"[2940, 2283]",...,0.019525,0,0.001144,0.005468,1,0.004188,0.008545,0.000000,0.000925,0.004296


In [27]:
colonies_bbox_psi = calc_all_services(colonies_bbox, point_services, line_services, epsg_code, 'nbrs_dist_bbox')

GeoDataFrame now has the following CRS:

epsg:7760
bank service index is completed
--------------------------------------------------------
GeoDataFrame now has the following CRS:

epsg:7760
health service index is completed
--------------------------------------------------------
GeoDataFrame now has the following CRS:

epsg:7760
police service index is completed
--------------------------------------------------------
GeoDataFrame now has the following CRS:

epsg:7760
ration service index is completed
--------------------------------------------------------
GeoDataFrame now has the following CRS:

epsg:7760
school service index is completed
--------------------------------------------------------
GeoDataFrame now has the following CRS:

epsg:7760
transport service index is completed
--------------------------------------------------------
all point services completed
GeoDataFrame now has the following CRS:

epsg:7760
road service is completed


In [31]:
colonies_bbox_psi = colonies_bbox_psi.rename(columns={'road_count':'road_length'})
colonies_bbox_psi.head()

,AREA,USO_AREA_U,HOUSETAX_C,USO_FINAL,geometry,canal,railway,drain,barrier,nbrs_bbox,...,ration_idx,school_count,school_pcen,school_idx,transport_count,transport_pcen,transport_idx,road_length,road_pcen,road_idx
0,Singhola,3058,H,RV,"POLYGON Z ((1013763.588 1023721.838 0.000, 101...",False,False,True,True,"[5108, 5487, 5488]",...,0.001925,1,0.000226,0.000843,8,0.001812,0.002876,5.957213,0.001349,0.005320
1,Indra Colony (Narela),1760,G,RUAC,"POLYGON Z ((1007997.730 1025421.961 0.000, 100...",False,True,False,True,"[2869, 5445]",...,0.000000,0,0.000380,0.001415,2,0.000849,0.001348,0.245861,0.000102,0.000401
2,Bhor Garh,1276,H,Industrial,"POLYGON Z ((1008543.236 1022671.585 0.000, 100...",False,True,False,True,"[4270, 4273, 5478]",...,0.000000,3,0.000438,0.001630,4,0.000584,0.000927,1.329173,0.000194,0.000765
3,Gautam Colony,1528,G,RUAC,"POLYGON Z ((1008080.674 1025132.190 0.000, 100...",False,True,False,True,"[2869, 3963, 4243, 5092, 5095, 5105, 5436, 544...",...,0.001749,7,0.000365,0.001356,7,0.000327,0.000519,0.827590,0.000037,0.000147
4,Kureni,2082,H,RV,"POLYGON Z ((1009508.695 1025281.671 0.000, 100...",False,True,False,True,"[3963, 5092, 5095, 5098, 5104, 5106, 5451, 5478]",...,0.000885,6,0.000208,0.000775,9,0.000341,0.000542,5.957236,0.000207,0.000815


## Save Files

In [36]:
colonies_bbox_psi.drop(columns=['nbrs_bbox', 'nbrs_dist_bbox', 'centroid']).to_file('delhi_psi_bbox_11Aug2020.shp')

In [37]:
colonies_touch_psi.drop(columns=['nbrs_touch', 'nbrs_dist_touch', 'centroid']).to_file('delhi_psi_touch_11Aug2020.shp')

In [38]:
colonies_bbox_psi.to_csv('delhi_psi_bbox_11Aug2020.csv')

In [39]:
colonies_touch_psi.to_csv('delhi_psi_touch_11Aug2020.csv')

In [40]:
with open('colonies_bbox_psi.data', 'wb') as f:
    pickle.dump(colonies_bbox_psi, f)

In [41]:
with open('colonies_touch_psi.data', 'wb') as f:
    pickle.dump(colonies_touch_psi, f)